# Intro. to Snorkel: Extracting Spouse Relations from the News

## Part IV: Training a Model with Data Programming

In this part of the tutorial, we will train a statistical model to differentiate between true and false `Spouse` mentions.

We will train this model using _data programming_, and we will **ignore** the training labels provided with the training data. This is a more realistic scenario; in the wild, hand-labeled training data is rare and expensive. Data programming enables us to train a model using only a modest amount of hand-labeled data for validation and testing. For more information on data programming, see the [NIPS 2016 paper](https://arxiv.org/abs/1605.07723).

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os,sys
os.environ['SNORKELDB']="postgres://jhusson@localhost:5432/snorkel_strom"

import numpy as np
from snorkel import SnorkelSession
session = SnorkelSession()

In [2]:
from snorkel.models import candidate_subclass

StromStrat = candidate_subclass('StromStrat', ['strom', 'stratname'])

We repeat our definition of the `Spouse` `Candidate` subclass from Parts II and III.

## Loading `CandidateSet` objects

We reload the training and development `CandidateSet` objects from the previous parts of the tutorial.

In [ ]:
from snorkel.models import CandidateSet

train = session.query(CandidateSet).filter(CandidateSet.name == 'News Training Candidates').one()
dev = session.query(CandidateSet).filter(CandidateSet.name == 'News Development Candidates').one()

## Automatically Creating Features
Recall that our goal is to distinguish between true and false mentions of spouse relations. To train a model for this task, we first embed our `Spouse` candidates in a feature space.

In [3]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()

We can create a new feature set:

In [16]:
%time F_train = feature_manager.create(session, train, 'Train Features')

[========================================] 100%=====                                  ] 13%

Loading sparse Feature matrix...
CPU times: user 2.07 s, sys: 154 ms, total: 2.23 s
Wall time: 3.2 s


**OR** if we've already created one, we can simply load as follows:

In [ ]:
%time F_train = feature_manager.load(session, train, 'Train Features')

Note that the returned matrix is a special subclass of the `scipy.sparse.csr_matrix` class, with some special features which we demonstrate below:

In [ ]:
F_train

In [ ]:
F_train.get_candidate(0)

In [ ]:
F_train.get_key(0)

## Creating Labeling Functions
Labeling functions are a core tool of data programming. They are heuristic functions that aim to classify candidates correctly. Their outputs will be automatically combined and denoised to estimate the probabilities of training labels for the training data.

In [32]:
import re
from snorkel.lf_helpers import is_inverted,get_left_tokens, get_right_tokens, get_between_tokens, get_text_between, get_tagged_text

## Applying Labeling Functions

First we construct a `LabelManager`.

In [5]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()

Next we run the `LabelManager` to to apply the labeling functions to the training `CandidateSet`.  We'll start with some of our labeling functions:

In [ ]:
spouses = {'wife', 'husband', 'ex-wife', 'ex-husband'}
family = {'father', 'mother', 'sister', 'brother', 'son', 'daughter',
              'grandfather', 'grandmother', 'uncle', 'aunt', 'cousin'}
family = family | {f + '-in-law' for f in family}
other = {'boyfriend', 'girlfriend' 'boss', 'employee', 'secretary', 'co-worker'}

def LF_too_far_apart(c):
    return -1 if len(get_between_tokens(c)) > 10 else 0

def LF_third_wheel(c):
    return -1 if 'PERSON' in get_between_tokens(c, attrib='ner_tags', case_sensitive=True) else 0
    
def LF_husband_wife(c):
    return 1 if len(spouses.intersection(set(get_between_tokens(c)))) > 0 else 0

def LF_husband_wife_left_window(c):
    if len(spouses.intersection(set(get_left_tokens(c[0], window=2)))) > 0:
        return 1
    elif len(spouses.intersection(set(get_left_tokens(c[1], window=2)))) > 0:
        return 1
    else:
        return 0

def LF_no_spouse_in_sentence(c):
    return -1 if len(spouses.intersection(set(c[0].parent.words))) == 0 else 0

def LF_and_married(c):
    return 1 if 'and' in get_between_tokens(c) and 'married' in get_right_tokens(c) else 0
    
def LF_familial_relationship(c):
    return -1 if len(set(family).intersection(set(get_between_tokens(c)))) > 0 else 0

def LF_family_left_window(c):
    if len(family.intersection(set(get_left_tokens(c[0], window=2)))) > 0:
        return -1
    elif len(family.intersection(set(get_left_tokens(c[1], window=2)))) > 0:
        return -1
    else:
        return 0

def LF_other_relationship(c):
    coworker = ['boss', 'employee', 'secretary', 'co-worker']
    return -1 if len(set(coworker).intersection(set(get_between_tokens(c)))) > 0 else 0

In [8]:
from snorkel.models import CandidateSet
all_c = session.query(CandidateSet).filter(CandidateSet.name == 'Candidate Set').one()

for c in all_c:
    if c[0].parent_id==11996:
        break

StromStrat(Span("stromatolites", parent=193044, chars=[285,297], words=[47,47]), Span("Smyrna Bed", parent=193044, chars=[0,9], words=[0,1]))


In [38]:
import yaml, psycopg2
from snorkel.models import Span

good_words={'strom':{'present','found'},'strat':{'contain','contains','include','includes'}}

# Connect to Postgres
with open('../credentials', 'r') as credential_yaml:
    credentials = yaml.load(credential_yaml)

with open('../config', 'r') as config_yaml:
    config = yaml.load(config_yaml)

# Connect to Postgres
connection = psycopg2.connect(
    dbname=credentials['snorkel_postgres']['database'],
    user=credentials['snorkel_postgres']['user'],
    password=credentials['snorkel_postgres']['password'],
    host=credentials['snorkel_postgres']['host'],
    port=credentials['snorkel_postgres']['port'])
cursor = connection.cursor()


def LF_num_stratphrase(c):
    cursor.execute("""
        SELECT distinct span.id from span 
        JOIN strom_strat on span.id=strom_strat.stratname_id  
        WHERE span.parent_id=%(parent_id)s;""",
                   {"parent_id": c[0].parent.id
                    })
    tmp_span=cursor.fetchall()

    tmp_strat = session.query(Span).filter(Span.id.in_(tmp_span)).all()
    num_strat = len({a.get_span() for a in tmp_strat})

    return -1 if num_strat > 1 else 1

test=LF_num_stratphrase(c)
print test

def LF_wordsep_fifty(c):
    return -1 if len(get_between_tokens(c)) > 50 else 0

test=LF_wordsep_fifty(c)
print test

def LF_wordsep_forty(c):
    return -1 if len(get_between_tokens(c)) > 40 else 0

test=LF_wordsep_forty(c)
print test

def LF_wordsep_thirty(c):
    return -1 if len(get_between_tokens(c)) > 30 else 0

test=LF_wordsep_thirty(c)
print test

def LF_wordsep_twenty(c):
    return -1 if len(get_between_tokens(c)) > 20 else 0

test=LF_wordsep_twenty(c)
print test

def LF_wordsep_ten(c):
    return -1 if len(get_between_tokens(c)) > 10 else 0

test=LF_wordsep_ten(c)
print test


def LF_nlp_parent(c):
    strom_parent = c[0].get_attrib_tokens('dep_parents')
    strom_idx = [c[0].get_word_start()+1,c[0].get_word_end()+1]

    strat_parent = c[1].get_attrib_tokens('dep_parents')
    strat_idx = [c[1].get_word_start()+1,c[1].get_word_end()+1]
    
    nlp_check = [True for a in strom_idx if a in strat_parent] + [True for a in strat_idx if a in strom_parent]
    return 0 if not nlp_check else 1

test=LF_nlp_parent(c)
print test

def LF_goodwords(c):
    if is_inverted(c):
        if len(good_words['strat'].intersection(set(get_between_tokens(c)))) > 0:
            return 1
        else:
            return 0
    else:
        if len(good_words['strom'].intersection(set(get_between_tokens(c)))) > 0:
            return 1
        else:
            return 0
        
test=LF_goodwords(c)
print test


1
0
0
0
0
0
0
1


In [13]:
LFs = [LF_num_stratphrase,LF_wordsep_fifty,LF_wordsep_forty,LF_wordsep_thirty,LF_wordsep_twenty,LF_wordsep_ten,LF_nlp_parent,LF_goodwords]

In [14]:
%time L_train = label_manager.create(session, train, 'LF Labels', f=LFs)
L_train

[========================================] 100%=====                                  ] 13%

Loading sparse Label matrix...
CPU times: user 381 ms, sys: 34.8 ms, total: 416 ms
Wall time: 613 ms


<15x7 sparse matrix of type '<type 'numpy.float64'>'
	with 26 stored elements in Compressed Sparse Row format>

**OR** load if we've already created:

In [ ]:
%time L_train = label_manager.load(session, train, 'LF Labels')
L_train

We can also add or rerun a single labeling function (or more!) with the below command. Note that we set the argument `expand_key_set` to `True` to indicate that the set of matrix columns should be allowed to expand:

In [ ]:
L_train = label_manager.update(session, train, 'LF Labels', True, f=[LF_no_spouse_in_sentence])
L_train

We can view statistics about the resulting label matrix:

In [15]:
L_train.lf_stats()

,conflicts,coverage,j,overlaps
LF_num_stratphrase,0,0.400000,0,0.333333
LF_wordsep_fifty,0,0.000000,1,0.000000
LF_wordsep_forty,0,0.000000,2,0.000000
LF_wordsep_thirty,0,0.000000,3,0.000000
LF_wordsep_twenty,0,0.133333,4,0.133333
LF_wordsep_ten,0,0.333333,5,0.333333
LF_nlp_parent,0,0.866667,6,0.533333


## Fitting the Generative Model
We estimate the accuracies of the labeling functions without supervision. Specifically, we estimate the parameters of a `NaiveBayes` generative model.

In [17]:
from snorkel.learning import NaiveBayes

gen_model = NaiveBayes()
gen_model.train(L_train, n_iter=1000, rate=1e-5)

//anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1318: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Training marginals (!= 0.5):	15
Features:			7
Begin training for rate=1e-05, mu=1e-06
	Learning epoch = 0	Gradient mag. = 0.526535
	Learning epoch = 250	Gradient mag. = 0.554394
	Learning epoch = 500	Gradient mag. = 0.554541
	Learning epoch = 750	Gradient mag. = 0.554688
Final gradient magnitude for rate=1e-05, mu=1e-06: 0.555


In [ ]:
gen_model.save(session, 'Generative Params')

We now apply the generative model to the training candidates.

In [18]:
train_marginals = gen_model.marginals(L_train)

In [19]:
gen_model.w

array([ 1.00078652,  0.99850038,  0.99850038,  0.99850038,  1.00003983,
        1.00180991,  1.00199087])

## Training the Discriminative Model
We use the estimated probabilites to train a discriminative model that classifies each `Candidate` as a true or false mention. We'll use a random hyperparameter search, evaluated on the development set labels, to find the best hyperparameters for our model. To run a hyperparameter search, we need labels for a development set. If they aren't already available, we can manually create labels using the Viewer.

In [21]:
from snorkel.learning import LogReg
from snorkel.learning_utils import RandomSearch, ListParameter, RangeParameter

iter_param = ListParameter('n_iter', [250, 500, 1000, 2000])
rate_param = RangeParameter('rate', 1e-4, 1e-2, step=0.75, log_base=10)
reg_param  = RangeParameter('mu', 1e-8, 1e-2, step=1, log_base=10)

disc_model = LogReg()

First, we create features for the development set candidates.

Note that we use the training feature set, because those are the only features for which we have learned parameters. Features that were not encountered during training, e.g., a token that does not appear in the training set, are ignored, because we do not have any information about them.

To do so with the `FeatureManager`, we call update with the new `CandidateSet`, the name of the training `AnnotationKeySet`, and the value `False` for the parameter `extend_key_set` to indicate that the `AnnotationKeySet` should not be expanded with new `Feature` keys encountered during processing.

In [22]:
%time F_dev = feature_manager.update(session, dev, 'Train Features', False)

[========================================] 100%===                                    ] 9%

Loading sparse Feature matrix...
CPU times: user 1.68 s, sys: 73.9 ms, total: 1.76 s
Wall time: 2.13 s


**OR** if we've already created one, we can simply load as follows:

In [ ]:
%time F_dev = feature_manager.load(session, dev, 'Train Features')

Next, we load the development set labels and gold candidates we made in Part III.

In [ ]:
L_gold_dev = label_manager.load(session, dev, "News Gold Labels")

In [ ]:
gold_dev_set = session.query(CandidateSet).filter(CandidateSet.name == 'News Development Candidates').one()

Now we set up and run the hyperparameter search, training our model with different hyperparamters and picking the best model configuration to keep. We'll set the random seed to maintain reproducibility.

Note that we are fitting our model's parameters to the training set generated by our labeling functions, while we are picking hyperparamters with respect to score over the development set labels which we created by hand.

In [23]:
searcher = RandomSearch(disc_model, F_train, train_marginals, 10, iter_param, rate_param, reg_param)

In [ ]:
np.random.seed(1701)
searcher.fit(F_dev, L_gold_dev, gold_dev_set)

_Note that to train a model without tuning any hyperparameters--at your own risk!--just use the `train` method of the discriminative model. For instance, to train with 500 iterations and a learning rate of 0.001, you could run:_
```
disc_model.train(F_train, train_marginals, n_iter=500, rate=0.001)
```

In [ ]:
disc_model.w.shape

In [ ]:
%time disc_model.save(session, "Discriminative Params")

In [ ]:
tp, fp, tn, fn = disc_model.score(F_dev, L_gold_dev, gold_dev_set)

## Viewing Examples
After evaluating on the development `CandidateSet`, the labeling functions can be modified. Try changing the labeling functions to improve performance. You can view the true positives, false positives, true negatives, and false negatives using the `Viewer`.

In [ ]:
from snorkel.viewer import SentenceNgramViewer

# NOTE: This if-then statement is only to avoid opening the viewer during automated testing of this notebook
# You should ignore this!
import os
if 'CI' not in os.environ:
    sv = SentenceNgramViewer(fn, session, annotator_name="Tutorial Part IV User")
else:
    sv = None

In [ ]:
sv

Next, in Part V, we will test our model on the test `CandidateSet`.